In [1]:
import sys
import importlib
import scanpy as sc
from pympler import asizeof
import numpy as np
sys.path.append('..')
import py8rds

# Get data

In [3]:
robj = py8rds.parse_rds("GSE268542_HuLu2023.RDS")

[2026-01-14 19:49:04,963][INFO] Reading GSE268542_HuLu2023.RDS
[2026-01-14 19:49:57,461][INFO] Done reading GSE268542_HuLu2023.RDS


In [4]:
robj.show()

IndexError: invalid index to scalar variable.

# Quick start
just read Seurat rds as adata

In [2]:
adata = py8rds.seurat2adata("GSE268542_HuLu2023.RDS")
adata

[2026-01-14 19:46:25,871][INFO] Reading GSE268542_HuLu2023.RDS
[2026-01-14 19:47:18,703][INFO] Done reading GSE268542_HuLu2023.RDS


AttributeError: 'NoneType' object has no attribute 'getClass'

In [ ]:
adata.obs

In [ ]:
sc.pl.embedding(adata,color='Cell.ID',basis='umap')

# A bit of customization
read file, check what is inside and select what to load

In [ ]:
robj = py8rds.parse_rds("GSM8704851_20210117-NMK0-FM1-Fc1U1Z1Bs1_obj.rds")
robj.show()

In [6]:
# by default seurat2adata takes first assay and 'counts' layer (slot), lets see whether we have anything else here:
# lets look on assays, we want to see slots in there, so lets change default number of levels to show
robj.get('assays').show(2)

IndexError: index 1 is out of bounds for axis 0 with size 0

In [ ]:
# so we can choose between counts and SCT assays
# inside them we can take norm data or counts
# instead of re-reading file we can pass robj to seurat2adata
# lets use second assay (SCTAssay) and normalized data from there
adata = py8rds.seurat2adata(robj,assay=1,layer='data')
adata

In [ ]:
sc.pl.embedding(adata,basis='umap',color=['nCount_Spatial','seurat_clusters'])

# Read me all
lets see a bit deeper into the object and parse individual parts of it
it might be useful for other rds files

In [ ]:
# lets look into SCTAssay
# there we have counts, data (norm counts) and even scale.data which are stored as dense matrix
robj.get(['assays',1]).show()

In [ ]:
# we can go further deep by adding indeces into argument of get function:
robj.get(['assays',1,'data','i']).show()

In [ ]:
# we can load any Matrix as anndata (is uses anndata not numpy in order to keep dimnames if any)
py8rds.as_anndata(robj.get(['assays',0,'data']))

In [ ]:
# as_anndata can handle dense arrays as well
py8rds.as_anndata(robj.get(['assays',1,'scale.data']))

In [ ]:
# or if you don't need anndata, lets load just array
py8rds.as_numpy(robj.get(['assays',1,'scale.data']))

In [ ]:
# or sparse ones:
py8rds.as_numpy(robj.get(['assays',0,'counts']))

In [ ]:
# or we can read data.frame
obs = py8rds.as_data_frame(robj.get('meta.data'))
obs

In [ ]:
# lets look on images 
robj.get(['images']).show()

In [ ]:
robj.get(['images',0]).show()

In [ ]:
robj.get(['images',0,'image']).show() # it is 3D array, lets load it

In [ ]:
# it is 3D array, lets load it
img = py8rds.as_numpy(robj.get(['images',0,'image']))
img

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(img)

# Load large 5G rds

In [ ]:
robj = py8rds.parse_rds("GSE232863_scRNA_omics.Rds")
robj.show()

[2026-01-14 20:27:06,922][INFO] Reading GSE232863_scRNA_omics.Rds


In [ ]:
robj.show()

In [ ]:
robj.get(['assays',1]).show(2)

In [ ]:
o = robj.get(['assays',1,'counts','i'])

In [ ]:
o.show()

In [ ]:
len(o.value)

In [ ]:
oo = o.value[1:100]

In [ ]:
size_gb = asizeof.asizeof(oo)
print(f"{size_gb:.6f} B")
print(f"{size_gb/2**10:.6f} KB")
print(f"{size_gb/2**20:.6f} MB")
print(f"{size_gb/2**30:.6f} GB")

In [ ]:
size_gb/100*211926780/2**30

In [ ]:
np.array(o.value)

In [ ]:
r = py8rds.parse_rds("GSM8704851_20210117-NMK0-FM1-Fc1U1Z1Bs1_obj.rds")

In [ ]:
z=np.zeros(10,dtype=np.int32)


In [ ]:
z[1] = np.nan

In [ ]:
z

In [ ]:
adata.X.data

In [ ]:
vars(adata.X)